## 분석목적: nusc 데이터 전처리 코드 분석

In [1]:
import argparse
import json
import os
import sys

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

import numpy as np
import torch
import yaml
from det3d.datasets import build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
import torch.distributed as dist
import subprocess

# 주피터 노트북 경로설정
os.chdir('../')


def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", default="configs/nusc/pp/nusc_centerpoint_pp_02voxel_two_pfn_10sweep.py", help="train config file path")
    parser.add_argument("--work_dir", help="the dir to save logs and models")
    parser.add_argument("--resume_from", help="the checkpoint file to resume from")
    parser.add_argument(
        "--validate",
        action="store_true",
        help="whether to evaluate the checkpoint during training",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument("--seed", type=int, default=None, help="random seed")
    parser.add_argument(
        "--launcher",
        choices=["pytorch", "slurm"],
        default="pytorch",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument(
        "--autoscale-lr",
        action="store_true",
        help="automatically scale lr with the number of gpus",
    )
    args = parser.parse_args(args=[])
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args
args = parse_args()

no apex
No Tensorflow
Deformable Convolution not built!
No APEX!


In [2]:
cfg = Config.fromfile(args.config)
cfg.local_rank = args.local_rank
distributed = False
logger = get_root_logger(cfg.log_level)
logger.info("Distributed training: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")

2024-03-19 13:19:08,995 - INFO - Distributed training: False
2024-03-19 13:19:08,996 - INFO - torch.backends.cudnn.benchmark: False


In [3]:
cfg.data.train

{'type': 'NuScenesDataset',
 'root_path': 'data/nuScenes',
 'info_path': 'data/nuScenes/infos_train_10sweeps_withvelo_filter_True.pkl',
 'ann_file': 'data/nuScenes/infos_train_10sweeps_withvelo_filter_True.pkl',
 'nsweeps': 10,
 'class_names': ['car',
  'truck',
  'construction_vehicle',
  'bus',
  'trailer',
  'barrier',
  'motorcycle',
  'bicycle',
  'pedestrian',
  'traffic_cone'],
 'pipeline': [{'type': 'LoadPointCloudFromFile', 'dataset': 'NuScenesDataset'},
  {'type': 'LoadPointCloudAnnotations', 'with_bbox': True},
  {'type': 'Preprocess',
   'cfg': {'mode': 'train',
    'shuffle_points': True,
    'global_rot_noise': [-0.3925, 0.3925],
    'global_scale_noise': [0.95, 1.05],
    'db_sampler': {'type': 'GT-AUG',
     'enable': False,
     'db_info_path': 'data/nuScenes/dbinfos_train_10sweeps_withvelo.pkl',
     'sample_groups': [{'car': 2},
      {'truck': 3},
      {'construction_vehicle': 7},
      {'bus': 4},
      {'trailer': 6},
      {'barrier': 2},
      {'motorcycle': 6}

## build_dataset

In [4]:
from det3d.datasets.registry import DATASETS

args = cfg.data.train.copy()
obj_type = args.pop("type")


In [5]:
for k, v in args.items():
    print(f"{k}")

root_path
info_path
ann_file
nsweeps
class_names
pipeline


In [6]:
obj_cls = DATASETS.get(obj_type)
default_args = dict(train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
if default_args is not None:
    for name, value in default_args.items():
        args.setdefault(name, value)
datasets = [obj_cls(**args)]

2024-03-19 13:19:16,604 - INFO - {'car': 5, 'truck': 5, 'bus': 5, 'trailer': 5, 'construction_vehicle': 5, 'traffic_cone': 5, 'barrier': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}
2024-03-19 13:19:16,604 - INFO - [-1]
2024-03-19 13:19:18,303 - INFO - load 62964 traffic_cone database infos
2024-03-19 13:19:18,303 - INFO - load 65262 truck database infos
2024-03-19 13:19:18,304 - INFO - load 339949 car database infos
2024-03-19 13:19:18,304 - INFO - load 161928 pedestrian database infos
2024-03-19 13:19:18,304 - INFO - load 26297 ignore database infos
2024-03-19 13:19:18,305 - INFO - load 11050 construction_vehicle database infos
2024-03-19 13:19:18,305 - INFO - load 107507 barrier database infos
2024-03-19 13:19:18,305 - INFO - load 8846 motorcycle database infos
2024-03-19 13:19:18,305 - INFO - load 8185 bicycle database infos
2024-03-19 13:19:18,305 - INFO - load 12286 bus database infos
2024-03-19 13:19:18,306 - INFO - load 19202 trailer database infos
2024-03-19 13:19:19,076

10


## NuScenesDataset

In [7]:
# obj_cls = DATASETS.get(obj_type) => det3d.datasets.nuscenes.nuscenes.NuScenesDataset

In [8]:
# __init__
import pickle
general_to_detection = {
    "human.pedestrian.adult": "pedestrian",
    "human.pedestrian.child": "pedestrian",
    "human.pedestrian.wheelchair": "ignore",
    "human.pedestrian.stroller": "ignore",
    "human.pedestrian.personal_mobility": "ignore",
    "human.pedestrian.police_officer": "pedestrian",
    "human.pedestrian.construction_worker": "pedestrian",
    "animal": "ignore",
    "vehicle.car": "car",
    "vehicle.motorcycle": "motorcycle",
    "vehicle.bicycle": "bicycle",
    "vehicle.bus.bendy": "bus",
    "vehicle.bus.rigid": "bus",
    "vehicle.truck": "truck",
    "vehicle.construction": "construction_vehicle",
    "vehicle.emergency.ambulance": "ignore",
    "vehicle.emergency.police": "ignore",
    "vehicle.trailer": "trailer",
    "movable_object.barrier": "barrier",
    "movable_object.trafficcone": "traffic_cone",
    "movable_object.pushable_pullable": "ignore",
    "movable_object.debris": "ignore",
    "static_object.bicycle_rack": "ignore",
}
def load_infos(info_path, class_names, load_interval, test_mode):

    with open(info_path, "rb") as f:
        _nusc_infos_all = pickle.load(f)

    _nusc_infos_all = _nusc_infos_all[::load_interval]

    if not test_mode:  # if training
        frac = int(len(_nusc_infos_all) * 0.25)

        _cls_infos = {name: [] for name in class_names}
        for info in _nusc_infos_all:
            for name in set(info["gt_names"]):
                if name in class_names:
                    _cls_infos[name].append(info)

        duplicated_samples = sum([len(v) for _, v in _cls_infos.items()])
        _cls_dist = {k: len(v) / max(duplicated_samples, 1) for k, v in _cls_infos.items()}

        _nusc_infos = []

        frac = 1.0 / len(class_names)
        ratios = [frac / v for v in _cls_dist.values()]

        for cls_infos, ratio in zip(list(_cls_infos.values()), ratios):
            _nusc_infos += np.random.choice(
                cls_infos, int(len(cls_infos) * ratio)
            ).tolist()

        _cls_infos = {name: [] for name in class_names}
        for info in _nusc_infos:
            for name in set(info["gt_names"]):
                if name in class_names:
                    _cls_infos[name].append(info)

        _cls_dist = {
            k: len(v) / len(_nusc_infos) for k, v in _cls_infos.items()
        }
    else:
        if isinstance(_nusc_infos_all, dict):
            _nusc_infos = []
            for v in _nusc_infos_all.values():
                _nusc_infos.extend(v)
        else:
            _nusc_infos = _nusc_infos_all
    
    return _nusc_infos

pipeline = args["pipeline"]
class_names = args["class_names"]
test_mode = False
version="v1.0-trainval"
load_interval = 1
nsweeps = args["nsweeps"]
root_path = args["root_path"]
assert nsweeps > 0
_info_path = args["info_path"]
_class_names = args["class_names"]
_num_point_features = 5
virtual = args.get('virtual', False)
eval_version = "detection_cvpr_2019"

_nusc_infos = load_infos(_info_path, _class_names, load_interval, test_mode)

logger.info(f"pipeline: {pipeline}")
logger.info(f"class_names: {class_names}")
logger.info(f"nsweeps: {nsweeps}")
logger.info(f"root_path: {root_path}")
logger.info(f"_info_path: {_info_path}")
logger.info(f"_class_names: {_class_names}")

2024-03-19 13:19:26,968 - INFO - pipeline: [{'type': 'LoadPointCloudFromFile', 'dataset': 'NuScenesDataset'}, {'type': 'LoadPointCloudAnnotations', 'with_bbox': True}, {'type': 'Preprocess', 'cfg': {'mode': 'train', 'shuffle_points': True, 'global_rot_noise': [-0.3925, 0.3925], 'global_scale_noise': [0.95, 1.05], 'db_sampler': {'type': 'GT-AUG', 'enable': False, 'db_info_path': 'data/nuScenes/dbinfos_train_10sweeps_withvelo.pkl', 'sample_groups': [{'car': 2}, {'truck': 3}, {'construction_vehicle': 7}, {'bus': 4}, {'trailer': 6}, {'barrier': 2}, {'motorcycle': 6}, {'bicycle': 6}, {'pedestrian': 2}, {'traffic_cone': 2}], 'db_prep_steps': [{'filter_by_min_num_points': {'car': 5, 'truck': 5, 'bus': 5, 'trailer': 5, 'construction_vehicle': 5, 'traffic_cone': 5, 'barrier': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}}, {'filter_by_difficulty': [-1]}], 'global_random_rotation_range_per_object': [0, 0], 'rate': 1.0}, 'class_names': ['car', 'truck', 'construction_vehicle', 'bus', 'trailer

In [9]:
logger.info(f"len(_nusc_infos): {len(_nusc_infos)}")
for k, v in _nusc_infos[0].items():
    logger.info(f"key of _nusc_infos: {k}")
_nusc_infos[0]

2024-03-19 13:19:26,974 - INFO - len(_nusc_infos): 123580
2024-03-19 13:19:26,975 - INFO - key of _nusc_infos: lidar_path
2024-03-19 13:19:26,975 - INFO - key of _nusc_infos: cam_front_path
2024-03-19 13:19:26,975 - INFO - key of _nusc_infos: cam_intrinsic
2024-03-19 13:19:26,975 - INFO - key of _nusc_infos: token
2024-03-19 13:19:26,975 - INFO - key of _nusc_infos: sweeps
2024-03-19 13:19:26,975 - INFO - key of _nusc_infos: ref_from_car
2024-03-19 13:19:26,976 - INFO - key of _nusc_infos: car_from_global
2024-03-19 13:19:26,976 - INFO - key of _nusc_infos: timestamp
2024-03-19 13:19:26,976 - INFO - key of _nusc_infos: all_cams_from_lidar
2024-03-19 13:19:26,976 - INFO - key of _nusc_infos: all_cams_intrinsic
2024-03-19 13:19:26,976 - INFO - key of _nusc_infos: all_cams_path
2024-03-19 13:19:26,976 - INFO - key of _nusc_infos: gt_boxes
2024-03-19 13:19:26,977 - INFO - key of _nusc_infos: gt_boxes_velocity
2024-03-19 13:19:26,977 - INFO - key of _nusc_infos: gt_names
2024-03-19 13:19:26

{'lidar_path': '/mnt/hdd0/nuScenes/samples/LIDAR_TOP/n008-2018-07-26-12-13-50-0400__LIDAR_TOP__1532622144198082.pcd.bin',
 'cam_front_path': '/mnt/hdd0/nuScenes/samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532622144162404.jpg',
 'cam_intrinsic': array([[1.25281310e+03, 0.00000000e+00, 8.26588115e+02],
        [0.00000000e+00, 1.25281310e+03, 4.69984663e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'token': '784d3895ca1c4bd1a1cdbe1d684a5299',
 'sweeps': [{'lidar_path': '/mnt/hdd0/nuScenes/sweeps/LIDAR_TOP/n008-2018-07-26-12-13-50-0400__LIDAR_TOP__1532622144148262.pcd.bin',
   'sample_data_token': 'aa8f4dd93e9742d79964851b2a520a95',
   'transform_matrix': array([[ 9.99999967e-01, -1.63876992e-04,  1.98193223e-04,
           -6.49580085e-04],
          [ 1.63976032e-04,  9.99999862e-01, -4.99800810e-04,
           -3.32687138e-01],
          [-1.98111290e-04,  4.99833293e-04,  9.99999855e-01,
           -1.34973613e-02],
          [ 0.00000000e+00,  0.

### compose 초기화

In [10]:
# Compose 
import collections
from det3d.utils import build_from_cfg
from det3d.datasets.registry import PIPELINES
class Compose(object):
    def __init__(self, transforms):
        assert isinstance(transforms, collections.abc.Sequence)
        self.transforms = []
        for transform in transforms:
            if isinstance(transform, dict):
                if transform['type'] == 'Empty':
                    continue 
                transform = build_from_cfg(transform, PIPELINES)
                self.transforms.append(transform)
            elif callable(transform):
                self.transforms.append(transform)
            else:
                raise TypeError("transform must be callable or a dict")

    def __call__(self, res, info):
        for t in self.transforms:
            res, info = t(res, info)
            if res is None:
                return None
        return res, info

    def __repr__(self):
        format_string = self.__class__.__name__ + "("
        for t in self.transforms:
            format_string += "\n"
            format_string += "    {0}".format(t)
        format_string += "\n)"
        return format_string

# _pipeline = Compose(pipeline)

### get_sensor_data

In [53]:
from pathlib import Path

idx = 0
info = _nusc_infos[idx]
_root_path = Path(root_path)
res = {
            "lidar": {
                "type": "lidar",
                "points": None,
                "nsweeps": nsweeps,
                # "ground_plane": -gp[-1] if with_gp else None,
                "annotations": None,
            },
            "metadata": {
                "image_prefix": _root_path,
                "num_point_features": _num_point_features,
                "token": info["token"],
            },
            "calib": None,
            "cam": {},
            "mode": "val" if test_mode else "train",
            "virtual": virtual 
        }

# data, _ = _pipeline(res, info)
# return data

In [54]:
# for k, v in data.items():
#     print(f"key of data: {k}")

### compose 분석

In [55]:
pipeline

[{'type': 'LoadPointCloudFromFile', 'dataset': 'NuScenesDataset'},
 {'type': 'LoadPointCloudAnnotations', 'with_bbox': True},
 {'type': 'Preprocess',
  'cfg': {'mode': 'train',
   'shuffle_points': True,
   'global_rot_noise': [-0.3925, 0.3925],
   'global_scale_noise': [0.95, 1.05],
   'db_sampler': {'type': 'GT-AUG',
    'enable': False,
    'db_info_path': 'data/nuScenes/dbinfos_train_10sweeps_withvelo.pkl',
    'sample_groups': [{'car': 2},
     {'truck': 3},
     {'construction_vehicle': 7},
     {'bus': 4},
     {'trailer': 6},
     {'barrier': 2},
     {'motorcycle': 6},
     {'bicycle': 6},
     {'pedestrian': 2},
     {'traffic_cone': 2}],
    'db_prep_steps': [{'filter_by_min_num_points': {'car': 5,
       'truck': 5,
       'bus': 5,
       'trailer': 5,
       'construction_vehicle': 5,
       'traffic_cone': 5,
       'barrier': 5,
       'motorcycle': 5,
       'bicycle': 5,
       'pedestrian': 5}},
     {'filter_by_difficulty': [-1]}],
    'global_random_rotation_range_

In [56]:
# Compose는 __call__에서 아래 type의 클래스를 순서대로 초기화함 
for i in pipeline:
    print(i['type'])

LoadPointCloudFromFile
LoadPointCloudAnnotations
Preprocess
Voxelization
AssignLabel
Reformat


In [57]:
# __init__
_transforms = []
for transform in pipeline:
    transform = build_from_cfg(transform, PIPELINES)
    _transforms.append(transform)

2024-03-19 16:03:34,696 - INFO - {'car': 5, 'truck': 5, 'bus': 5, 'trailer': 5, 'construction_vehicle': 5, 'traffic_cone': 5, 'barrier': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}
2024-03-19 16:03:34,698 - INFO - [-1]
2024-03-19 16:03:36,710 - INFO - load 62964 traffic_cone database infos
2024-03-19 16:03:36,710 - INFO - load 65262 truck database infos
2024-03-19 16:03:36,711 - INFO - load 339949 car database infos
2024-03-19 16:03:36,711 - INFO - load 161928 pedestrian database infos
2024-03-19 16:03:36,711 - INFO - load 26297 ignore database infos
2024-03-19 16:03:36,712 - INFO - load 11050 construction_vehicle database infos
2024-03-19 16:03:36,712 - INFO - load 107507 barrier database infos
2024-03-19 16:03:36,712 - INFO - load 8846 motorcycle database infos
2024-03-19 16:03:36,712 - INFO - load 8185 bicycle database infos
2024-03-19 16:03:36,712 - INFO - load 12286 bus database infos
2024-03-19 16:03:36,713 - INFO - load 19202 trailer database infos
2024-03-19 16:03:37,563

In [58]:
# __call__(데이터를 불러 들일때 마다 실행)
import time
s = time.time()
for t in _transforms:
    s = time.time()
    res, info = t(res, info)
    e = time.time() - s
    print(e)

0.011544942855834961
2.8848648071289062e-05
0.3583996295928955
0.0075991153717041016
0.0037679672241210938
1.049041748046875e-05


### 순서대로 실행
1. LoadPointCloudFromFile
2. LoadPointCloudAnnotations
3. Preprocess
4. Voxelization
5. AssignLabel
6. Reformat

In [16]:
# 초기 res
res

{'lidar': {'type': 'lidar',
  'points': None,
  'nsweeps': 10,
  'annotations': None},
 'metadata': {'image_prefix': PosixPath('data/nuScenes'),
  'num_point_features': 5,
  'token': '784d3895ca1c4bd1a1cdbe1d684a5299'},
 'calib': None,
 'cam': {},
 'mode': 'train',
 'virtual': False}

In [17]:
# 초기 info (= _nusc_infos[idx])
info

{'lidar_path': '/mnt/hdd0/nuScenes/samples/LIDAR_TOP/n008-2018-07-26-12-13-50-0400__LIDAR_TOP__1532622144198082.pcd.bin',
 'cam_front_path': '/mnt/hdd0/nuScenes/samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532622144162404.jpg',
 'cam_intrinsic': array([[1.25281310e+03, 0.00000000e+00, 8.26588115e+02],
        [0.00000000e+00, 1.25281310e+03, 4.69984663e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 'token': '784d3895ca1c4bd1a1cdbe1d684a5299',
 'sweeps': [{'lidar_path': '/mnt/hdd0/nuScenes/sweeps/LIDAR_TOP/n008-2018-07-26-12-13-50-0400__LIDAR_TOP__1532622144148262.pcd.bin',
   'sample_data_token': 'aa8f4dd93e9742d79964851b2a520a95',
   'transform_matrix': array([[ 9.99999967e-01, -1.63876992e-04,  1.98193223e-04,
           -6.49580085e-04],
          [ 1.63976032e-04,  9.99999862e-01, -4.99800810e-04,
           -3.32687138e-01],
          [-1.98111290e-04,  4.99833293e-04,  9.99999855e-01,
           -1.34973613e-02],
          [ 0.00000000e+00,  0.

In [18]:
# LoadPointCloudFromFile
loadPointCloudFromFileInstance = _transforms[0]
res, info = loadPointCloudFromFileInstance(res, info)

In [19]:
res

{'lidar': {'type': 'lidar',
  'points': array([[ -3.0795703 ,  -0.31610546,  -1.8388232 ,   8.        ],
         [ -3.257438  ,  -0.3144277 ,  -1.8426977 ,  11.        ],
         [ -3.4738622 ,  -0.31309614,  -1.8595222 ,   9.        ],
         ...,
         [-14.155887  ,  -0.32315767,   0.97915715,  39.        ],
         [-14.162879  ,  -0.31836632,   1.3106116 ,  33.        ],
         [-14.165775  ,  -0.3160578 ,   1.64586   ,  15.        ]],
        dtype=float32),
  'nsweeps': 10,
  'annotations': None,
  'times': array([[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.04981995],
         [0.04981995],
         [0.04981995]], dtype=float32),
  'combined': array([[ -3.0795703 ,  -0.31610546,  -1.8388232 ,   8.        ,
            0.        ],
         [ -3.257438  ,  -0.3144277 ,  -1.8426977 ,  11.        ,
            0.        ],
         [ -3.4738622 ,  -0.31309614,  -1.8595222 ,   9.        ,
            0.        ],
         ...,
    

In [20]:
# LoadPointCloudAnnotations
LoadPointCloudAnnotationsInstance = _transforms[1]
res, info = LoadPointCloudAnnotationsInstance(res, info)

In [21]:
res

{'lidar': {'type': 'lidar',
  'points': array([[ -3.0795703 ,  -0.31610546,  -1.8388232 ,   8.        ],
         [ -3.257438  ,  -0.3144277 ,  -1.8426977 ,  11.        ],
         [ -3.4738622 ,  -0.31309614,  -1.8595222 ,   9.        ],
         ...,
         [-14.155887  ,  -0.32315767,   0.97915715,  39.        ],
         [-14.162879  ,  -0.31836632,   1.3106116 ,  33.        ],
         [-14.165775  ,  -0.3160578 ,   1.64586   ,  15.        ]],
        dtype=float32),
  'nsweeps': 10,
  'annotations': {'boxes': array([[ 5.71997261e+00, -8.97246933e+00, -1.05986071e+00,
            6.90999985e-01,  1.04499996e+00,  1.90400004e+00,
            2.09998131e-01,  1.45142329e+00, -3.03907394e+00],
          [ 5.78404379e+00, -1.82107658e+01, -1.50393748e+00,
            6.42000020e-01,  6.49999976e-01,  1.64499998e+00,
           -3.30765471e-02,  2.00397763e-02, -1.12695992e+00],
          [ 7.80096102e+00, -3.43064537e+01, -2.31470251e+00,
            1.75000000e+00,  4.37400007e+00,

In [22]:
# Preprocess
PreprocessInstance = _transforms[2]
res, info = PreprocessInstance(res, info)

In [23]:
res

{'lidar': {'type': 'lidar',
  'points': array([[  9.182673  ,   3.6302805 ,  -0.6951361 ,   4.        ,
            0.10011101],
         [ -4.9129643 ,  -2.1698394 ,  -1.6967909 ,   5.        ,
            0.34949207],
         [ -1.5324616 ,  -3.1721377 ,  -1.6555088 ,   7.        ,
            0.19953394],
         ...,
         [ 12.993329  ,  -7.2969036 ,  -1.5493572 ,   6.        ,
            0.29973292],
         [  1.577883  , -21.15203   ,  -0.7657958 ,  25.        ,
            0.44959402],
         [ -1.9893194 ,   5.0923347 ,  -1.9785517 ,   9.        ,
            0.39688396]], dtype=float32),
  'nsweeps': 10,
  'annotations': {'gt_boxes': array([[ 8.48306179e+00,  6.04852676e+00, -1.03774309e+00,
            6.76579893e-01,  1.02319241e+00,  1.86426651e+00,
           -3.41553450e-01, -1.39471936e+00,  6.56451797e+00],
          [ 1.19287205e+01,  1.44122934e+01, -1.47255266e+00,
            6.28602505e-01,  6.36435509e-01,  1.61067140e+00,
           -3.73776816e-02, -6

In [26]:
# Voxelization
VoxelizationInstance = _transforms[3]
res, info = VoxelizationInstance(res, info)

2024-03-19 14:24:11,742 - INFO - finding looplift candidates


In [27]:
res

{'lidar': {'type': 'lidar',
  'points': array([[  9.182673  ,   3.6302805 ,  -0.6951361 ,   4.        ,
            0.10011101],
         [ -4.9129643 ,  -2.1698394 ,  -1.6967909 ,   5.        ,
            0.34949207],
         [ -1.5324616 ,  -3.1721377 ,  -1.6555088 ,   7.        ,
            0.19953394],
         ...,
         [ 12.993329  ,  -7.2969036 ,  -1.5493572 ,   6.        ,
            0.29973292],
         [  1.577883  , -21.15203   ,  -0.7657958 ,  25.        ,
            0.44959402],
         [ -1.9893194 ,   5.0923347 ,  -1.9785517 ,   9.        ,
            0.39688396]], dtype=float32),
  'nsweeps': 10,
  'annotations': {'gt_boxes': array([[ 8.48306179e+00,  6.04852676e+00, -1.03774309e+00,
            6.76579893e-01,  1.02319241e+00,  1.86426651e+00,
           -3.41553450e-01, -1.39471936e+00,  6.56451797e+00],
          [ 1.19287205e+01,  1.44122934e+01, -1.47255266e+00,
            6.28602505e-01,  6.36435509e-01,  1.61067140e+00,
           -3.73776816e-02, -6

In [28]:
# AssignLabel
AssignLabelInstance = _transforms[4]
res, info = AssignLabelInstance(res, info)

In [29]:
res

{'lidar': {'type': 'lidar',
  'points': array([[  9.182673  ,   3.6302805 ,  -0.6951361 ,   4.        ,
            0.10011101],
         [ -4.9129643 ,  -2.1698394 ,  -1.6967909 ,   5.        ,
            0.34949207],
         [ -1.5324616 ,  -3.1721377 ,  -1.6555088 ,   7.        ,
            0.19953394],
         ...,
         [ 12.993329  ,  -7.2969036 ,  -1.5493572 ,   6.        ,
            0.29973292],
         [  1.577883  , -21.15203   ,  -0.7657958 ,  25.        ,
            0.44959402],
         [ -1.9893194 ,   5.0923347 ,  -1.9785517 ,   9.        ,
            0.39688396]], dtype=float32),
  'nsweeps': 10,
  'annotations': {'gt_boxes': [array([[ 1.9661789e+01,  2.8285677e+01, -2.2663984e+00,  1.7134802e+00,
             4.2827215e+00,  1.4833844e+00,  0.0000000e+00,  0.0000000e+00,
             3.2855082e-01],
           [ 1.2923565e+01,  4.0037661e+00, -1.0553857e+00,  2.0219069e+00,
             4.7243099e+00,  1.8397882e+00,  6.1540317e-04,  2.1467833e-03,
        